# 1- Bibliotecas Usadas ( Comumente)

In [3]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import scipy as sp
import statsmodels.api as sm
import patsy as pt
from sklearn import metrics
from sklearn.model_selection import train_test_split
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.linear_model import Ridge

from sklearn.metrics import mean_absolute_error

# 2- EDA -

## 2.1 -Estatísticas descritivas 

In [ ]:
#df.describe()
#df.isnull().sum
#df.info()

### 2.1.1- Plot de correlação

In [2]:
plot = sns.heatmap(corrWM, annot = True, cmap="RdBu", fmt=".2f", linewidths=.6)
plot

NameError: name 'corrWM' is not defined

## 2.2- Plotar Varios Histogramas

In [6]:
    ##Nome das colunas - Pode somente numericas
#contColNames = list(df.select_dtypes(include='number').columns)
    ##Tamnho subplot
#ncols = 3
#nrows = int(np.ceil(len(contColNames)/(1.0 * ncols)))

In [ ]:
##Plot
fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(20,10))

counter = 0 

for i in range(nrows):
  for j in range(ncols):

    ax = axes[i][j]

    # Plotar somente quando houver dados
    if counter < len(contColNames):

      ax.hist(df.select_dtypes(include='number')[contColNames[counter]], bins=10)
      ax.set_xlabel(contColNames[counter])
      ax.set_ylabel('Frequência')

    else:
      ax.set_axis_off()

    counter += 1

plt.show()

## 2.3- PairPlot

In [4]:
sns.pairplot(df)

NameError: name 'df' is not defined

In [ ]:
df = 

## 2.4- Plota Varios BoxPlot

In [5]:
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec

def draw_boxplots(df, selected_features):
  n = len(selected_features)
  fig = plt.figure(constrained_layout=True, figsize=(20, 30))
  gs = gridspec.GridSpec(int(n/3) + 1, 3, figure=fig)
  for i in range(n):
    k = int(i / 3)
    j = i % 3
    col = selected_features[i]
    ax = fig.add_subplot(gs[k, j])
    ax.set_title(col)
    ax.boxplot(df[col])

draw_boxplots(df_train, df_train.columns)

NameError: name 'df_train' is not defined

## 2.5- Remove Auto Outliers

In [ ]:
def replace_outliers(df, var_names):
  for var_name in var_names:
    # calculating Q1 and Q3
    Q1 = df.loc[:,var_name].quantile(0.25)
    Q3 = df.loc[:,var_name].quantile(0.75)
    # calculating Interquartile IQR
    IQR = Q3 - Q1
    # build up filter
    filter_up = (df.loc[:,var_name] > Q3 + 1.5 *IQR)
    # apply up filter on df
    df.loc[filter_up, var_name] = Q3 + 1.5 *IQR
    # build down filter
    filter_down = (df.loc[:,var_name] < Q1 - 1.5 * IQR)
    # apply down filter on df
    df.loc[filter_down, var_name] = Q1 - 1.5 * IQR

# 3 PRÉ-PROCESSAMENTO DOS DADOS

## 3.1- Selecionar os tipos de variaveis.

### 3.1.1- Number e Category

In [ ]:
#Seleciona o tipo de variavel Number dentro do data
dfNum = df.select_dtypes(include='number').drop(['Unnamed: 0'], axis = 1)
dfNum.head()

In [ ]:
# Mudar o tipo de variável para 'categórico'
df['cut'] = df['cut'].astype('category')
df['color'] = df['color'].astype('category')
df['clarity'] = df['clarity'].astype('category')
# Selecionar somente as variáveis categóricas
dfCat = df.select_dtypes(include='category')
dfCat.head(2)

### 3.1.2- Variavel Target

In [ ]:
y = df['varTarget']

### 3.2.3 Teste log da resposta

In [ ]:
kStat, kPvalue = sp.stats.kstest(rvs=np.log(y), cdf='norm')

In [ ]:
kStat

In [ ]:
kPvalue

## 3.2 - Escalonamento da variavel

In [ ]:
# Criar a função para o escalonamento dos dados
def feat_scale(X):

  # Calcular a média de todas as variáveis
  mu = np.mean(X, axis = 0)

  # Calcular o devio padrão de todas as variáveis
  sigma = np.std(X, axis=0, ddof=1)

  # Fazer os escalonamento das variáveis
  xNorm = (X - mu)/sigma

  return xNorm

In [ ]:
#Atribuição ao DF
dfNorm = feat_scale(dfNum)
dfNorm.head(2)

In [ ]:
# Normalização das variáveis- Com Sklearn
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler()
scaler.fit(X_train)

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Transformand para dataframe para visualização ## O Xtrain viara Array
X_train = pd.DataFrame(X_train_scaled,columns = X_train.columns)
X_test = pd.DataFrame(X_test_scaled,columns = X_test.columns)

## 3.3 -Pré-tratamento das variáveis categóricas 

### 3.3.1- Dummificação

In [ ]:
dfDum = pd.get_dummies(dfCat)
dfDum.head(2)

## 3.4 - Concatenação do Data 

In [8]:
dfFinal = pd.concat([dfNorm.reset_index(drop=True), dfDum], axis=1)
dfFinal.head(2)

NameError: name 'dfNorm' is not defined

### 3.4.1 - Com OneHotEncoder Scklearn

In [ ]:
from sklearn.preprocessing import OneHotEncoder

enc = OneHotEncoder(handle_unknown='ignore')
enc.fit(X)
#Criando a Lista
list(zip(X, enc.transform(X).toarray()))

#Transformando e Trainando#
X_transformed = enc.transform(X).toarray()

reg = LinearRegression().fit(X_transformed, y)

# Retorna o coeficiente de determinação R² da previsão:
# melhor quando mais perto de zero
reg.score(X_transformed, y)

# 5 - Separa o Data em Train e Teste

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42) # o 0,33 significa 33% dos dados
print(X_train.shape)
print(X_test.shape)

## 5.1- Ajuste Modelo OLS

In [ ]:
import statsmodels.api as sm
reg3 = sm.OLS(yTrain, xTrain)
reg3fit = reg3.fit()
print(reg1fit.summary())

# 6 Predição

## 6.1  Regressão Linear

In [13]:
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error,mean_squared_error,r2_score
reg = LinearRegression().fit(X_train, y_train)

y_train_pred = reg.predict(X_train)
y_pred = reg.predict(X_test)

print("MAE = {}".format(mean_absolute_error(y_test, y_pred)))
print("RMSE = {}".format(mean_squared_error(y_test, pred, squared=False)))
print("R² = {}".format(r2_score(y_test, y_pred)))

NameError: name 'X_train' is not defined

# 6 - Hiperparemetros

In [ ]:
from sklearn.model_selection import GridSearchCV

parameters = {'max_depth': [2, 5, 10], 'n_estimators':[10, 100, 300]}
regr = RandomForestRegressor(max_depth=5, random_state=0)
clf = GridSearchCV(regr, parameters)
clf.fit(X_train_preprocessed, y_train_preprocessed)

In [ ]:
y_pred = clf.predict(X_test_preprocessed)
mean_absolute_error(y_test_preprocessed, y_pred)

In [ ]:
r2_score(y_test_preprocessed, y_pred)

In [ ]:
clf.best_params_

## 7 -métricas de avaliação de qualidade

In [ ]:

from sklearn.metrics import accuracy_score, f1_score, recall_score, precision_score
print(f'Acurácia: {accuracy_score(y_test.squeeze(), y_pred) * 100:.3f} % \n'
      f'Precisão: {precision_score(y_test.squeeze(), y_pred) * 100:.3f} % \n'
      f'Recall:   {recall_score(y_test.squeeze(), y_pred) * 100:.3f} % \n'
      f'F1:       {f1_score(y_test.squeeze(), y_pred) * 100:.3f} %')

# 10 Gráficos Tops

In [10]:
import matplotlib.pyplot as plt
plt.scatter(X, y, edgecolors='k', c=['r'])
plt.xlabel('Feature 1')
plt.ylabel('Prices')
plt.title('Prices Boston')

plt.plot(X, y_pred, marker = 'o')
plt.show()

NameError: name 'X' is not defined